## 문장 기반한 데이터셋 만들기

앞서, 게시판에 재학님이 올리신 것을 바탕으로 모두가 동일한 문장을 제거하면 될 것 같습니다.

저는 그 이후 문장이 중복되는 경우 하나의 데이터셋으로만 흘러가도록 만드는 것을 구현하고자 했습니다!

우선 loda_data.py와 train에 있는 label_to_num을 활용해 데이터를 불러오고     
categorical labelling을 수행해주었습니다.

In [15]:
from load_data import *
from train import label_to_num

all_dataset = load_data("../dataset/train/train.csv")
all_label = label_to_num(all_dataset['label'].values)

all_dataset.head(5)

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation


정답 레이블의 경우에는 list 형태로 되어 있었습니당

In [16]:
all_label[:5]

[0, 0, 20, 1, 0]

중복되는 문장 확인을 위한 is_duplicated column을 만들어주었습니다!

In [17]:
all_dataset['is_duplicated'] = all_dataset['sentence'].duplicated(keep=False)
all_dataset.head(5)

,id,sentence,subject_entity,object_entity,label,is_duplicated
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation,False
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation,False
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of,False
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees,False
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation,False


label의 경우에도 일단은 DataFrame으로 만들어주었습니다

In [18]:
all_label = pd.DataFrame(train_label, columns=['label'])
all_label.head(5)

,label
0,0
1,0
2,20
3,1
4,0


그리고 is_duplicated를 기반으로 데이터셋을 나누어주었습니다.    
이를 바탕으로 코드 작성이 진행되었습니다.

In [19]:
import sklearn
from sklearn.model_selection import train_test_split


not_duplicated_data = all_dataset[all_dataset['is_duplicated'] == False]
not_duplicated_label = all_label.iloc[all_dataset[all_dataset['is_duplicated'] == False].index]

duplicated_data = all_dataset[all_dataset['is_duplicated'] == True]
duplicated_label = all_label.iloc[all_dataset[all_dataset['is_duplicated'] == True].index]


### 초창기 버전

처음 버전은 50% 확률로 train 혹은 valid 데이터셋이 선택되어 거기 있는 모든 중복 데이터를 다른 곳으로 옮기는 방식입니다.
이 방식을 사용하면 한 번은 train에 모두 중복되지 않은 데이터, valid에는 중복된 데이터 + 중복되지 않은 데이터 일정량     
두번째는 반대로 진행되게 됩니다.

하지만 이런식으로 사용하면 train이나 valid에서 문제가 생길 것 같아 보류로 해두었습니다.

In [20]:
import random
from sklearn.model_selection import StratifiedKFold
kfold= StratifiedKFold(n_splits= 5, shuffle= True, random_state= 42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(all_dataset, all_label)):
    train_dataset= all_dataset.iloc[train_idx]
    val_dataset= all_dataset.iloc[val_idx]
    train_label= all_label.iloc[train_idx]
    val_label= all_label.iloc[val_idx]
    
    train_dataset.reset_index(drop= True, inplace= True)
    val_dataset.reset_index(drop= True, inplace= True)
    
    temp = []    
    if random.random() > 0.5:
        for val_idx in val_dataset.index:
            if val_dataset['is_duplicated'].iloc[val_idx] == True:
                train_dataset.append(val_dataset.iloc[val_idx])
                temp.append(val_idx)
        val_dataset.drop(temp, inplace= True, axis= 0)
    else:
        for train_idx in train_dataset.index:
            if train_dataset['is_duplicated'].iloc[train_idx] == True:
                val_dataset.append(train_dataset.iloc[train_idx])
                temp.append(train_idx)
        train_dataset.drop(temp, inplace= True, axis= 0)

    
    
    print(len(train_dataset), len(val_dataset))

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


20271 6494
20307 6494
20276 6494
25976 5083
20369 6494


분할은 그래도 나쁘지 않게 되는 모습

### 보완 버전

해당 버전에서는 valid를 기반으로 진행됩니다    
먼저, 문장들을 읽으며 is_duplicated인 경우에 중복되는 문장이 train에 있는지 확인합니     
만약 train에 있는 경우 한 곳으로 몰아주기 위해 데이터를 train으로 옮겨줍니다.    
그리고 해당 idx를 temp에 저장한 이후에 valid 데이터를 모두 처리한 후 drop을 시켜줍니다.

In [23]:
import random
kfold = StratifiedKFold(n_splits= 5, shuffle= True, random_state= 42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(all_dataset, all_label)):
    train_dataset= all_dataset.loc[train_idx]
    val_dataset= all_dataset.loc[val_idx]
    train_dataset.reset_index(drop= True, inplace= True)
    val_dataset.reset_index(drop= True, inplace= True)
    
    temp = []    
    
    for val_idx in val_dataset.index:
        if val_dataset['is_duplicated'].loc[val_idx] == True:
            if val_dataset['sentence'].loc[val_idx] in train_dataset['sentence'].values:
                train_dataset.append(val_dataset.loc[val_idx])
                temp.append(val_idx)
            
    val_dataset.drop(temp, inplace= True, axis= 0)

    print(len(train_dataset), len(val_dataset))


25976 5339
25976 5314
25976 5355
25976 5337
25976 5256


분할은 위와 같이 깔끔하게 잘 됩니다.. 근데 왜 에러 날까요ㅠㅠ

## 아래는 아직 작업중인 코드들입니다!

In [ ]:
def label_to_num(label):
  num_label = []
  with open('/opt/ml/git/level2-klue-level2-nlp-03/eunki/code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

In [ ]:
from load_data import *

total_train_dataset = load_data("../dataset/train/train.csv")
total_train_label = label_to_num(total_train_dataset['label'].values)
pd.DataFrame(data = total_train_label, columns=['label']).reset_index(drop= True)

In [ ]:
from sklearn.model_selection import train_test_split

train_test_split(train_dataset, train_label, test_size=0.2, random_state=42, stratify=[train_dataset['label'], train_dataset['sentence']])

In [ ]:
class Preprocess:
    def __init__(self, CSV_PATH, version):
        self.version= version
        self.data= self.load_data(CSV_PATH)

    def load_data(self, path):
        
        data= pd.read_csv(path)
        
        sub_entity, sub_type= [], []
        obj_entity, obj_type= [], []
        sub_idx, obj_idx= [], []
        sentence= []

        """preprocess"""
        for i, [x, y, z] in enumerate(zip(data['subject_entity'], data['object_entity'], data['sentence'])):
            sub_typ= x[1:-1].split(':')[-1].split('\'')[-2]
            obj_typ= y[1:-1].split(':')[-1].split('\'')[-2]
            
            for idx_i in range(len(x)):
                if x[idx_i: idx_i+ 9]== 'start_idx':
                    sub_start= int(x[idx_i+12:].split(',')[0].strip())
                if x[idx_i: idx_i+7]== 'end_idx':
                    sub_end= int(x[idx_i+10:].split(',')[0].strip())
                
                if y[idx_i: idx_i+ 9]== 'start_idx':
                    obj_start= int(y[idx_i+12:].split(',')[0].strip())
                if y[idx_i: idx_i+7]== 'end_idx':
                    obj_end= int(y[idx_i+10:].split(',')[0].strip())
            
            sub_i= [sub_start, sub_end]
            obj_i= [obj_start, obj_end]

            sub_entity.append(z[sub_i[0]: sub_i[1]+1])
            obj_entity.append(z[obj_i[0]: obj_i[1]+1])
            sub_type.append(sub_typ); sub_idx.append(sub_i)
            obj_type.append(obj_typ); obj_idx.append(obj_i)
            
            """tokenize version"""
            # if self.version== 'SUB':
            #     if sub_i[0] < obj_i[0]:
            #         z= z[:sub_i[0]] + '[SUB]'+ z[sub_i[0]: sub_i[1]+1] + '[/SUB]' + z[sub_i[1]+1:]
            #         z= z[:obj_i[0]+11] + '[OBJ]'+ z[obj_i[0]+11: obj_i[1]+12]+ '[/OBJ]'+ z[obj_i[1]+12:]
            #     else:
            #         z= z[:obj_i[0]] + '[OBJ]'+ z[obj_i[0]: obj_i[1]+1]+ '[/OBJ]'+ z[obj_i[1]+1:]
            #         z= z[:sub_i[0]+11] + '[SUB]'+ z[sub_i[0]+11: sub_i[1]+12] + '[/SUB]' + z[sub_i[1]+12:]

            # elif self.version== 'PUN':
            #     if sub_i[0] < obj_i[0]:
            #         z= z[:sub_i[0]] + '@*'+sub_typ+'*'+ z[sub_i[0]: sub_i[1]+1] + '@' + z[sub_i[1]+1:]
            #         z= z[:obj_i[0]+7] + '#^'+ obj_typ +'^'+ z[obj_i[0]+7: obj_i[1]+8]+ '#'+ z[obj_i[1]+8:]
            #     else:
            #         z= z[:obj_i[0]] + '#^'+ obj_typ +'^'+ z[obj_i[0]: obj_i[1]+1]+ '#' + z[obj_i[1]+1:]
            #         z= z[:sub_i[0]+7] + '@*'+sub_typ+'*' + z[sub_i[0]+7: sub_i[1]+8] + '@' + z[sub_i[1]+8:]

            sentence.append(z)

        df= pd.DataFrame({'id': data['id'], 'sentence' : sentence, 'subject_entity': sub_entity, 'object_entity': obj_entity,
                                'subject_type': sub_type, 'object_type': obj_type, 'label': data['label'],
                                'subject_idx': sub_idx, 'object_idx': obj_idx})
        
        return df

In [ ]:
preprocess= Preprocess("../dataset/train/train.csv", 'SUB')

In [ ]:
preprocessing_dataset = preprocess.data